In [1]:

import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\i518410\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\i518410\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\i518410\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\i518410\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\i518410\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\i518410\AppDat

True

In [21]:
import pandas as pd
import numpy as np

In [2]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews


# A function that extracts which words exist in a text based on a list of words to which we compare.
def word_feats(words):
        return dict([(word, True) for word in words])

In [6]:
negids = movie_reviews.fileids('neg')

# Get the positive reviews for movies
posids = movie_reviews.fileids('pos')
 
# Find the features that most correspond to negative reviews    
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]

# Find the features that most correspond to positive reviews
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# We would only use 1500 instances to train on. The quarter of the reviews left is for testing purposes.
negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)

In [32]:
votes= pd.read_csv(r"C:\Users\i518410\OneDrive - SAP SE\Documents\UvA DATA SCIENCE\Fundamentals of Data Science\Scripts and Datasets\Assignment1\usa-2016-presidential-election-by-county.csv", error_bad_lines=False, sep=";")

In [33]:
votes.head()

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),...,total08,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2
0,Alaska,AK,2100,"Haines Borough, Alaska",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nebraska,NE,31145,"Red Willow County, Nebraska",13.0,5061.0,1216.0,952.0,3735.0,3891.0,...,5044.0,4936.0,93.0,93.0,0.018841,0.018438,0.803428,0.754393,0.196572,0.245607
2,California,CA,6015,"Del Norte County, California",18.0,9361.0,4323.0,3791.0,4967.0,4614.0,...,9531.0,8770.0,241.0,365.0,0.041619,0.025286,0.548959,0.534661,0.451041,0.465339
3,Florida,FL,12031,"Duval County, Florida",199.0,428734.0,202618.0,196737.0,210537.0,211615.0,...,415761.0,411848.0,2606.0,3496.0,0.008489,0.006268,0.518217,0.509584,0.481783,0.490416
4,Kansas,KS,20053,"Ellsworth County, Kansas",25.0,2626.0,851.0,702.0,2021.0,1930.0,...,2938.0,2712.0,66.0,80.0,0.029499,0.022464,0.733283,0.703691,0.266717,0.296309


In [34]:
votes_state= votes.groupby(by=['State'], as_index=False)[["Votes", "Votes16 Trumpd", "Votes16 Clintonh"]].sum()

In [9]:
votes["State"]

0               Alaska
1             Nebraska
2           California
3              Florida
4               Kansas
             ...      
3138         Tennessee
3139          Michigan
3140     West Virginia
3141    North Carolina
3142          Maryland
Name: State, Length: 3143, dtype: object

In [35]:
votes_state

,State,Votes,Votes16 Trumpd,Votes16 Clintonh
0,Alabama,2078165.0,1306925.0,718084.0
1,Alaska,0.0,0.0,0.0
2,Arizona,2062810.0,1021154.0,936250.0
3,Arkansas,1121684.0,677904.0,378729.0
4,California,11954317.0,3916209.0,7362490.0
5,Colorado,2564185.0,1137455.0,1212209.0
6,Connecticut,1623542.0,668266.0,884432.0
7,Delaware,441535.0,185103.0,235581.0
8,District of Columbia,280272.0,11553.0,260223.0
9,Florida,9386750.0,4605515.0,4485745.0


In [36]:
votes_state["Trump %"]= (votes_state["Votes16 Trumpd"]/votes_state["Votes"])*100

In [25]:
votes_state

,State,Votes,Votes16 Trumpd,Votes16 Clintonh,Trump/Clinton,Trump %,Clinton %
0,Alabama,2078165.0,1306925.0,718084.0,62.888414,62.888414,34.553753
1,Alaska,0.0,0.0,0.0,NaN,NaN,NaN
2,Arizona,2062810.0,1021154.0,936250.0,49.503057,49.503057,45.387118
3,Arkansas,1121684.0,677904.0,378729.0,60.436273,60.436273,33.764322
4,California,11954317.0,3916209.0,7362490.0,32.759789,32.759789,61.588546
5,Colorado,2564185.0,1137455.0,1212209.0,44.359319,44.359319,47.274631
6,Connecticut,1623542.0,668266.0,884432.0,41.160992,41.160992,54.475462
7,Delaware,441535.0,185103.0,235581.0,41.922611,41.922611,53.355000
8,District of Columbia,280272.0,11553.0,260223.0,4.122067,4.122067,92.846592
9,Florida,9386750.0,4605515.0,4485745.0,49.064000,49.064000,47.788052


In [37]:
votes_state["Clinton %"]= (votes_state["Votes16 Clintonh"]/votes_state["Votes"])*100 

In [20]:
votes_state["T or H"]=
for i in votes_state:
    if votes_state["Trump %"] > votes_state["Clinton %"]:
        votes_state["T or H"] =="T"
    else:
        votes_state["T or H"] =="H"
        

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [38]:
votes_state['T or H'] = np.where(votes_state["Trump %"] > votes_state["Clinton %"], "Trump", "Hillary")

In [40]:
votes_state= votes_state.drop([1])

In [41]:
votes_state

,State,Votes,Votes16 Trumpd,Votes16 Clintonh,Trump %,Clinton %,T or H
0,Alabama,2078165.0,1306925.0,718084.0,62.888414,34.553753,Trump
2,Arizona,2062810.0,1021154.0,936250.0,49.503057,45.387118,Trump
3,Arkansas,1121684.0,677904.0,378729.0,60.436273,33.764322,Trump
4,California,11954317.0,3916209.0,7362490.0,32.759789,61.588546,Hillary
5,Colorado,2564185.0,1137455.0,1212209.0,44.359319,47.274631,Hillary
6,Connecticut,1623542.0,668266.0,884432.0,41.160992,54.475462,Hillary
7,Delaware,441535.0,185103.0,235581.0,41.922611,53.355000,Hillary
8,District of Columbia,280272.0,11553.0,260223.0,4.122067,92.846592,Hillary
9,Florida,9386750.0,4605515.0,4485745.0,49.064000,47.788052,Trump
10,Georgia,4029564.0,2068623.0,1837300.0,51.336150,45.595504,Trump


In [43]:
votes_state.to_csv('US_2016_winners.csv', index=False)